# Building My Agentic AI Application In Snowflake

### *NOTE: For overview, prerequisites and other instructions, refer to this [guide](https://github.com/Snowflake-Labs/sfguide-build-data-agents-using-snowflake-cortex-ai).*


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

## Setup Tools

### Cortex Search: Tool for Unstructured Data

Setup a tool that will help the agent to extract information from unstructured data. It will process PDF documents about bikes and skis, and also use image descriptions. The information is stored in PDF and JPEG format.

### PDFs

In [ ]:
-- Preview documents
SELECT * FROM DIRECTORY('@DOCS');

Read/process the PDF files using SNOWFLAKE.CORTEX.PARSE_DOCUMENT

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE RAW_TEXT AS
SELECT 
    RELATIVE_PATH,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
            '@DOCS',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'} ):content
        ) AS EXTRACTED_LAYOUT 
FROM 
    DIRECTORY('@DOCS')
WHERE
    RELATIVE_PATH LIKE '%.pdf';

In [ ]:
select * from RAW_TEXT limit 5;

Create the table that will be used by Cortex Search service as a tool for Cortex Agents in order to retrieve information from PDF and JPEG files

In [ ]:
create or replace TABLE DOCS_CHUNKS_TABLE ( 
    RELATIVE_PATH VARCHAR(16777216), -- Relative path to the PDF file
    CHUNK VARCHAR(16777216), -- Piece of text
    CHUNK_INDEX INTEGER, -- Index for the text
    CATEGORY VARCHAR(16777216) -- Will hold the document category to enable filtering
);


In [ ]:
insert into DOCS_CHUNKS_TABLE (relative_path, chunk, chunk_index)

    select relative_path, 
            c.value::TEXT as chunk,
            c.INDEX::INTEGER as chunk_index
            
    from 
        raw_text,
        LATERAL FLATTEN( input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
              EXTRACTED_LAYOUT,
              'markdown',
              1512,
              256,
              ['\n\n', '\n', ' ', '']
           )) c;

In [ ]:
SELECT * FROM DOCS_CHUNKS_TABLE limit 7;


Let's see how CLASSIFY_TEXT Cortex function can be used to classify the document type. We have two classes, Bike and Snow, and we pass the document title and the first chunk of the document to the function.

In [ ]:
CREATE OR REPLACE TEMPORARY TABLE docs_categories AS WITH unique_documents AS (
  SELECT
    DISTINCT relative_path, chunk
  FROM
    docs_chunks_table
  WHERE 
    chunk_index = 0
  ),
 docs_category_cte AS (
  SELECT
    relative_path,
    TRIM(snowflake.cortex.CLASSIFY_TEXT (
      'Title:' || relative_path || 'Content:' || chunk, ['Bike', 'Snow']
     )['label'], '"') AS category
  FROM
    unique_documents
)
SELECT
  *
FROM
  docs_category_cte;

In [ ]:
select * from docs_categories;


In [ ]:
update docs_chunks_table 
  SET category = docs_categories.category
  from docs_categories
  where  docs_chunks_table.relative_path = docs_categories.relative_path;


### Images

Now let's process the images we have for our bikes and skis. We are going to use COMPLETE multi-modal function to generate image descriptions and classifications. We will add this information to the DOCS_CHUNKS_TABLE where we also have the PDF documentation.

In [ ]:
insert into DOCS_CHUNKS_TABLE (relative_path, chunk, chunk_index, category)
SELECT 
    RELATIVE_PATH,
    CONCAT('This is a picture describing the bike or ski: '|| RELATIVE_PATH || 
        'THIS IS THE DESCRIPTION: ' ||
        SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet',
        'DESCRIBE THIS IMAGE: ',
        TO_FILE('@DOCS', RELATIVE_PATH))) as chunk,
    0,
    SNOWFLAKE.CORTEX.COMPLETE('claude-3-5-sonnet',
        'Classify this image, respond only with Bike or Snow: ',
        TO_FILE('@DOCS', RELATIVE_PATH)) as category,
FROM 
    DIRECTORY('@DOCS')
WHERE
    RELATIVE_PATH LIKE '%.jpeg';


In [ ]:
select * from DOCS_CHUNKS_TABLE
    where RELATIVE_PATH LIKE '%.jpeg';

### Cortex Search

Cortex Search tool will be used to retrieve context from unstructured data. Once we have processed all the content from PDFs and images into the DOCS_CHUNK_TABLE, we just need to enable the service in that table. This will automatically create the embeddings, indexing, etc. 

In [ ]:
create or replace CORTEX SEARCH SERVICE DOCUMENTATION_TOOL
ON chunk
ATTRIBUTES relative_path, category
warehouse = COMPUTE_WH
TARGET_LAG = '1 hour'
EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
as (
    select chunk,
        chunk_index,
        relative_path,
        category
    from docs_chunks_table
);

### Cortex Analyst: Tool for Structured Data

Another tool that we will setup is Cortex Analyst. It will provide the capability to extract information from structured data stored in Snowflake tables. In the API call we will provider the location of our semantic file that contains information about the business terminology used to describe the data.

First, let's create some tables and generate data that provides additional context about Robot results.

**ROBOT_RESULTS – Structural Analysis Results Table**

**Purpose:**  
Stores stress results for structural members as exported from Autodesk Robot Structural Analysis.  
This data helps engineers evaluate structural performance under different load cases.

**Key Columns:**

- MEMBER_NODE_CASE (Primary Key): Combined string identifying the member number, node number, and load case (e.g. `1/194/100 (C)`).
- S_MAX_MPA: Maximum stress value in MPa.
- S_MIN_MPA: Minimum stress value in MPa.
- S_MAX_MY_MPA: Maximum bending stress about the local y-axis in MPa.
- S_MAX_MZ_MPA: Maximum bending stress about the local z-axis in MPa.
- S_MIN_MY_MPA: Minimum bending stress about the local y-axis in MPa.
- S_MIN_MZ_MPA: Minimum bending stress about the local z-axis in MPa.
- FX_AX_MPA: Axial stress in MPa.






In [ ]:
-- Create table for structural analysis results
CREATE OR REPLACE TABLE DIM_STRUCTURAL_ANALYSIS (
    ANALYSIS_ID INT AUTOINCREMENT PRIMARY KEY,
    MEMBER_ID INT,
    NODE_ID INT,
    CASE_ID STRING,
    CASE_TYPE STRING, -- 'BASIC' or 'COMBINATION' (for cases with 'C' suffix)
    S_MAX_MPA FLOAT,
    S_MIN_MPA FLOAT,
    S_MAX_MY_MPA FLOAT,
    S_MAX_MZ_MPA FLOAT,
    S_MIN_MY_MPA FLOAT,
    S_MIN_MZ_MPA FLOAT,
    FX_AX_MPA FLOAT,
    MEMBER_NODE_CASE STRING -- Original identifier for reference
);

-- Insert sample data from your structural analysis
INSERT INTO DIM_STRUCTURAL_ANALYSIS (
    MEMBER_ID, NODE_ID, CASE_ID, CASE_TYPE, S_MAX_MPA, S_MIN_MPA, 
    S_MAX_MY_MPA, S_MAX_MZ_MPA, S_MIN_MY_MPA, S_MIN_MZ_MPA, 
    FX_AX_MPA, MEMBER_NODE_CASE
) VALUES 
-- Member 1, Node 194 cases
(1, 194, '1', 'BASIC', 4.87, -10.39, 5.88, 0.86, -5.88, -2.64, -1.87, '1/194/1'),
(1, 194, '2', 'BASIC', 0.02, -0.05, 0.03, 0.00, -0.03, -0.02, -0.01, '1/194/2'),
(1, 194, '3', 'BASIC', 0.09, -0.07, 0.07, 0.01, -0.07, -0.02, 0.02, '1/194/3'),
(1, 194, '10', 'BASIC', 0.03, -0.02, 0.02, 0.00, -0.02, -0.00, 0.00, '1/194/10'),
(1, 194, '11', 'BASIC', 1.58, -4.27, 2.52, 0.20, -2.52, -0.62, -1.13, '1/194/11'),
(1, 194, '12', 'BASIC', 4.71, -1.08, 2.45, 0.67, -2.45, -0.22, 1.59, '1/194/12'),
(1, 194, '13', 'BASIC', 2.00, -0.64, 0.25, 1.61, -0.25, -0.52, 0.14, '1/194/13'),
(1, 194, '14', 'BASIC', 0.45, -1.57, 0.26, 0.37, -0.26, -1.13, -0.18, '1/194/14'),
(1, 194, '100', 'COMBINATION', 9.14, -20.61, 11.88, 1.47, -11.88, -4.52, -4.21, '1/194/100 (C)'),
(1, 194, '101', 'COMBINATION', 5.14, -7.14, 4.42, 0.84, -4.42, -2.59, -0.13, '1/194/101 (C)'),
(1, 194, '102', 'COMBINATION', 6.58, -11.97, 8.49, 0.38, -8.49, -1.18, -2.30, '1/194/102 (C)'),
(1, 194, '103', 'COMBINATION', 6.65, -15.77, 7.71, 1.72, -7.71, -5.29, -2.78, '1/194/103 (C)'),
(1, 194, '200', 'COMBINATION', 6.59, -14.79, 8.52, 1.07, -8.52, -3.28, -2.99, '1/194/200 (C)'),
(1, 194, '201', 'COMBINATION', 3.92, -5.81, 3.55, 0.65, -3.55, -1.99, -0.27, '1/194/201 (C)'),
(1, 194, '202', 'COMBINATION', 4.88, -9.03, 6.26, 0.34, -6.26, -1.06, -1.72, '1/194/202 (C)'),
(1, 194, '203', 'COMBINATION', 4.93, -11.57, 5.74, 1.23, -5.74, -3.79, -2.04, '1/194/203 (C)'),
(1, 194, '1000', 'BASIC', 6.58, -14.02, 7.94, 1.16, -7.94, -3.56, -2.52, '1/194/1000'),

-- Member 1, Node 347 cases
(1, 347, '1', 'BASIC', 3.73, -4.32, 3.33, 1.05, -3.33, -0.34, -0.65, '1/347/1'),
(1, 347, '2', 'BASIC', -0.05, -0.08, 0.01, 0.00, -0.01, -0.01, -0.06, '1/347/2'),
(1, 347, '3', 'BASIC', 0.04, 0.01, 0.01, 0.01, -0.01, -0.00, 0.02, '1/347/3'),
(1, 347, '10', 'BASIC', 0.02, -0.01, 0.01, 0.01, -0.01, -0.00, 0.01, '1/347/10'),
(1, 347, '11', 'BASIC', -0.62, -0.82, 0.04, 0.03, -0.04, -0.08, -0.69, '1/347/11'),
(1, 347, '12', 'BASIC', 1.17, 0.90, 0.06, 0.10, -0.06, -0.03, 1.00, '1/347/12'),
(1, 347, '13', 'BASIC', 0.25, -0.47, 0.27, 0.04, -0.27, -0.14, -0.07, '1/347/13'),
(1, 347, '14', 'BASIC', 0.34, -0.32, 0.26, 0.11, -0.26, -0.03, -0.02, '1/347/14'),
(1, 347, '100', 'COMBINATION', 3.82, -6.85, 4.47, 1.30, -4.47, -0.42, -1.95, '1/347/100 (C)'),
(1, 347, '101', 'COMBINATION', 6.79, -4.56, 4.64, 1.57, -4.64, -0.51, 0.58, '1/347/101 (C)'),
(1, 347, '102', 'COMBINATION', 4.33, -5.54, 4.14, 1.21, -4.14, -0.39, -1.01, '1/347/102 (C)'),
(1, 347, '103', 'COMBINATION', 5.55, -6.40, 4.93, 1.57, -4.93, -0.51, -0.95, '1/347/103 (C)'),
(1, 347, '200', 'COMBINATION', 2.92, -5.00, 3.32, 0.97, -3.32, -0.32, -1.37, '1/347/200 (C)'),
(1, 347, '201', 'COMBINATION', 4.90, -3.48, 3.43, 1.15, -3.43, -0.37, 0.32, '1/347/201 (C)'),
(1, 347, '202', 'COMBINATION', 3.26, -4.13, 3.09, 0.91, -3.09, -0.30, -0.74, '1/347/202 (C)'),
(1, 347, '203', 'COMBINATION', 4.07, -4.70, 3.62, 1.15, -3.62, -0.37, -0.70, '1/347/203 (C)'),
(1, 347, '1000', 'BASIC', 5.04, -5.84, 4.50, 1.42, -4.50, -0.46, -0.88, '1/347/1000');

-- You can add more rows following the same pattern for the rest of your data
-- Note: Rows with N/A values should be excluded or handled with NULL values

-- Create some useful views for analysis
CREATE OR REPLACE VIEW STRESS_SUMMARY AS
SELECT 
    MEMBER_ID,
    NODE_ID,
    CASE_TYPE,
    MAX(S_MAX_MPA) as MAX_STRESS_MPA,
    MIN(S_MIN_MPA) as MIN_STRESS_MPA,
    AVG(ABS(S_MAX_MPA)) as AVG_ABS_MAX_STRESS,
    COUNT(*) as NUM_CASES
FROM DIM_STRUCTURAL_ANALYSIS 
WHERE S_MAX_MPA IS NOT NULL
GROUP BY MEMBER_ID, NODE_ID, CASE_TYPE;

CREATE OR REPLACE VIEW CRITICAL_STRESS_POINTS AS
SELECT 
    MEMBER_NODE_CASE,
    MEMBER_ID,
    NODE_ID,
    CASE_ID,
    CASE_TYPE,
    S_MAX_MPA,
    S_MIN_MPA,
    ABS(S_MAX_MPA) as ABS_MAX_STRESS,
    ABS(S_MIN_MPA) as ABS_MIN_STRESS
FROM DIM_STRUCTURAL_ANALYSIS 
WHERE ABS(S_MAX_MPA) > 10 OR ABS(S_MIN_MPA) > 10
ORDER BY ABS_MAX_STRESS DESC;

: 

### Semantic Models

The semantic model maps business terminology to the structured data and adds contextual meaning. It allows Cortex Analyst to generate the correct SQL for a question asked in natural language.

NOTE: To explore semantic models, refer to these [instructions](https://github.com/Snowflake-Labs/sfguide-build-data-agents-using-snowflake-cortex-ai?tab=readme-ov-file#step-5-explore-the-semantic-model).

In [ ]:
create or replace stage semantic_files ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') DIRECTORY = ( ENABLE = true );
#changed database with my own
COPY FILES
    INTO @semantic_files/
    FROM @SOPHIE_CORTEX_AGENTS_SUMMIT.PUBLIC.git_repo/branches/main/
    FILES = ('semantic.yaml', 'semantic_search.yaml');

### Improve Tool Usage with Dynamic Literal Retrieval

Using Cortex Analyst integration with Cortex Search, we can improve the retrieval of possible values of a column without listing them all in the semantic model file. 

Let's try it as example for the ARTICLE NAMES.

In [ ]:
CREATE OR REPLACE CORTEX SEARCH SERVICE _ARTICLE_NAME_SEARCH
  ON ARTICLE_NAME
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 hour'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT ARTICLE_NAME
  FROM DIM_ARTICLE
);

### *NOTE: This concludes the setup tools portion of this guide. Follow [instructions outlined here](https://quickstarts.snowflake.com/guide/build-agentic-application-in-snowflake/index.html?index=..%2F..index#3) to proceed to next steps.*